In [ ]:
from pathlib import Path

import molli

molli.visual.configure(bgcolor="#1F1F1F")

root_dir = Path("notebooks/qsar")

xyzs = list(sorted(Path("data/dft/qsar/benzylamine").rglob("step4.xtbopt.xyz")))

catalysts_mlib = molli.library.MoleculeLibrary(
    root_dir / "catalysts.mlib",
    readonly=False,
    overwrite=True,
)

with catalysts_mlib.writing():
    for xyz in xyzs:
        mol = molli.load(xyz, parser="obabel")
        mol.name = xyz.parent.stem

        catalysts_mlib[mol.name] = mol


In [38]:
with catalysts_mlib.reading():
    mol = catalysts_mlib["L-148"]

mol

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Molecule(name='L-148', formula='C73 H91 F6 N2 O8 P1 Pd1')

In [43]:
from molli.pipeline.xtb import XTBDriver

properties_mlib = molli.library.MoleculeLibrary(
    root_dir / "catalysts_prop.mlib",
    readonly=False,
    overwrite=True,
)

xtb = XTBDriver(nprocs=8, executable="/home/mgcf/software-ws/xtb/xtb-6.7.1/bin/xtb")

molli.pipeline.jobmap(
    xtb.atom_properties_m,
    source=catalysts_mlib,
    destination=properties_mlib,
    cache_dir="/scr/nhadler/",
    scratch_dir="/scr/nhadler/",
    n_workers=2,
    kwargs={
        "method": "gfn2",
        "misc": "--alpb thf",
    },
    progress=True,
    verbose=True,
)


Starting a molli.pipeline.jobmap_sge calculation:
calculation: XTBDriver.atom_properties_m
None
     source:  MoleculeLibrary(backend=UkvCollectionBackend('notebooks/qsar/catalysts.mlib'), n_items=125)
destination:  MoleculeLibrary(backend=UkvCollectionBackend('notebooks/qsar/catalysts_prop.mlib'), n_items=0)
scratch dir:  /scr/nhadler
Total number of jobs:      125
Exist in destination:        0
To be computed:            125


Finalizing the calculations: 100%|██████████| 125/125 [00:09<00:00, 12.67it/s]


In [49]:
with properties_mlib.reading():
    mol = properties_mlib["L-148"]

mol.atoms[0].attrib

{'symbol': 'Pd',
 'disp': '287.329',
 'pol': '34.958',
 'charge': '-0.363',
 'covCN': '3.104',
 'f+': '0.144',
 'f-': '0.317',
 'f0': '0.231',
 'max_bond_order': '0.943'}